In [1]:
import pandas as pd 
import numpy as np
import datetime 
import time
pd.set_option('display.max_columns', None)
day=time.strftime("%Y-%m-%d")

# 运行前需确认和输入的信息

##  【手动输入】原始数据文件位置

In [2]:
file_location = r'C:\Users\Administrator\Desktop\无车_好运宝钱包5.0-2020.07.07.xlsx'

## 【手动输入】上次统计的日期

In [3]:
# 手动输入上次做表的日期数字
before_last_week = datetime.date(2020,6,29).strftime('%Y%m%d')

## 【手动输入】总部和区域销售员数据的文件位置

In [4]:
salesman_project_week_location = r'C:\Users\Administrator\Desktop\赵惠中\销售员项目数.xlsx'

## 【输入】导出文件位置

In [5]:
wallet_detail_file_location = "E:\\"+"好运宝钱包明细"+day+".xlsx"

# 生成好运宝钱包完成情况总表

## 【导入】数据-今日产生有效钱包数

In [6]:
# 读取今日产生有效钱包数
num_of_wallet_today = pd.read_excel(file_location,sheet_name='今日产生有效钱包数',dtype = {'好运宝钱包配置开启时间':'datetime64'})

In [7]:
num_of_wallet_today.loc[num_of_wallet_today['部门']=='实施运营部','部门']='商务运营部'

In [8]:
# 筛选列（企业id，企业名称，部门，销售姓名
num_of_wallet_today_2 = num_of_wallet_today[['企业ID','企业名称','部门','销售姓名']].copy()

In [9]:
# 去重
num_of_wallet_today_2.drop_duplicates(inplace=True)

In [10]:
# 汇总每个企业累计的有效钱包数
group1 = num_of_wallet_today.groupby('企业ID')['今日产生有效钱包数'].sum()

In [11]:
# 转换为Dataframe
comp_id_wallet_num = pd.DataFrame({'企业ID':group1.index,'累计有效钱包数':group1.values})

In [12]:
# 拼接企业，部门，销售，钱包数
comp_id_wallet_num_2 = pd.merge(num_of_wallet_today_2,comp_id_wallet_num,how='left',on='企业ID')

## 【导入】数据-承运司机汇总（每个企业ID对应的承运司机数

In [13]:
# 读取企业关联的承运司机数量信息
driver_num = pd.read_excel(file_location,sheet_name='承运司机汇总')

In [14]:
driver_num_2 = driver_num[['企业ID','承运司机的数量']].copy()

## 【导入】支付汇总（配置了好运宝钱包的企业ID 对应的支付运单信息

In [15]:
# 读取企业运单信息
pay_num = pd.read_excel(file_location,sheet_name='支付汇总',dtype = {'好运宝钱包配置开启时间':'datetime64'})

In [16]:
# 筛选出需求列
pay_num_2 = pay_num[['企业ID','支付运单数量', '本人收款运单数量',
       '付运费到钱包的运单数量', '付运费到钱包的司机数量', '支付单里的司机数', '支付到本人的司机数']].copy()

In [17]:
# 拼接企业ID对应的司机数
comp_id_wallet_num_3 = comp_id_wallet_num_2.merge(driver_num_2,how='left',left_on='企业ID',right_on='企业ID').copy()

In [18]:
# 拼接企业ID对应的运单信息
comp_id_wallet_num_4 = comp_id_wallet_num_3.merge(pay_num_2,how='left',left_on='企业ID',right_on='企业ID').copy()

In [19]:
# 新增计算列，支付比例
comp_id_wallet_num_4['支付比例'] = comp_id_wallet_num_4['付运费到钱包的运单数量']/comp_id_wallet_num_4['支付运单数量']

## 筛选上次统计日期前的数据

In [20]:
# 筛选出上周的数据
num_of_wallet_before_last_week = num_of_wallet_today[num_of_wallet_today['日期']<before_last_week].copy()

In [21]:
# 每个企业ID上周累计的钱包数汇总
group_2 = num_of_wallet_before_last_week.groupby('企业ID')['今日产生有效钱包数'].sum()

In [22]:
# 转换为表格
comp_id_wallet_num_before = pd.DataFrame({'企业ID':group_2.index,'上周累计有效钱包数':group_2.values})

In [23]:
# 拼接上周每个企业对应的钱包数和部门销售
comp_id_wallet_num_before_2 = pd.merge(comp_id_wallet_num_before,num_of_wallet_today_2,how='left',on='企业ID')

In [24]:
# 上周每个部门累计的钱包数汇总
before_lastweek_wallet_num = comp_id_wallet_num_before_2.groupby('部门')['上周累计有效钱包数'].sum()

In [25]:
before_lastweek_wallet_num_1 = pd.DataFrame({'部门':before_lastweek_wallet_num.index,'上周钱包数汇总':before_lastweek_wallet_num.values})

In [26]:
# 每个部门对应的企业ID数量
before_lastweek_comp_num = comp_id_wallet_num_before_2.groupby('部门')['企业ID'].count()

In [27]:
# 转换为表格
before_lastweek_pay_num = pd.DataFrame({'部门':before_lastweek_comp_num.index,'上周已支付企业数汇总':before_lastweek_comp_num.values})

## 透视得到目前每个部门的数据

In [28]:
# 透视每个部门下的数据汇总
group_3 = comp_id_wallet_num_4.groupby('部门').agg({'企业ID':'count','支付运单数量':'sum',
                                        '承运司机的数量':'sum','本人收款运单数量':'sum',
                                        '付运费到钱包的运单数量':'sum','累计有效钱包数':'sum'})

In [29]:
# 重置索引
wallet_wancheng = group_3.reset_index(inplace=True)

In [30]:
# 修改列名
group_3.rename({'企业ID':'已支付企业数汇总','支付运单数量':'支付运单总数','承运司机的数量':'承运司机总数',
               '本人收款运单数量':'本人收款运单数','付运费到钱包的运单数量':'支付到钱包运单数','累计有效钱包数':'钱包数汇总'},axis='columns',inplace=True)

In [31]:
# 拼接部门对应的上周支付企业数
group_4 = group_3.merge(before_lastweek_pay_num,how='left',on='部门')

In [32]:
# 新增字段
group_4['新增支付企业数'] = group_4['已支付企业数汇总'] - group_4['上周已支付企业数汇总']

In [33]:
# 拼接每个部门对应的上周钱包数
group_4 = group_4.merge(before_lastweek_wallet_num_1,how='left',on='部门')

In [34]:
# 新增字段
group_4['周增长'] = group_4['钱包数汇总'] - group_4['上周钱包数汇总']

In [35]:
# 新增字段
group_4['钱包支付比例（50%支付比例）']=group_4['支付到钱包运单数']/group_4['支付运单总数']

In [36]:
# 删除拼接的上周数据
group_4.drop(['上周已支付企业数汇总','上周钱包数汇总'],axis=1,inplace=True)

In [37]:
dept_list = ['营销一部','营销二部','营销三部','营销四部','营销五部','营销六部','营销七部','营销八部','培训营',
                '集团一部','集团二部','集团三部','集团四部','商务运营部','区域营销部']

group_4['部门'] = pd.Categorical(group_4['部门'], categories=dept_list, ordered=True)

group_4.sort_values(by='部门',inplace=True)

## 【导出】为excel文件-好运宝钱包明细+日期.xlsx

In [38]:
# 导出为EXCEL
group_4.to_excel(wallet_detail_file_location,index=False)

# 生成总明细表

In [39]:
total_project = num_of_wallet_today_2.merge(driver_num_2,how='left',on='企业ID')

In [40]:
total_project = total_project.merge(comp_id_wallet_num,how='left',on='企业ID')

In [41]:
total_project = total_project.merge(comp_id_wallet_num_before,how='left',on='企业ID')

In [42]:
total_project['周增长'] = total_project['累计有效钱包数'] - total_project['上周累计有效钱包数']

In [43]:
total_project = total_project.merge(pay_num_2,how='left',on='企业ID')

In [44]:
total_project['支付比例'] = total_project['付运费到钱包的运单数量']/total_project['支付运单数量']

# 生成区域和总部销售表

## 已开通项目的相关数据

In [45]:
# 获取销售原始数据
data_salesperson = total_project.copy()

## 【导入】区域和总部的销售人员数据

In [46]:
# 导入区域销售
area_salesperson = pd.read_excel(salesman_project_week_location,sheet_name='区域')
# 导入总部销售
general_salesperson = pd.read_excel(salesman_project_week_location,sheet_name='总部')

## 生成区域销售数据

In [47]:
# 每个销售名下开通钱包服务的项目数量
dept_num = data_salesperson.groupby('销售姓名')['企业ID'].count()
# 转换
dept_num_1 = pd.DataFrame({'销售姓名':dept_num.index,'开通钱包项目数':dept_num.values})
# 拼接
area_salesperson_data = area_salesperson.merge(dept_num_1,how='left',on='销售姓名')

In [48]:
# 新增计算列
area_salesperson_data['开通钱包项目率'] = area_salesperson_data['开通钱包项目数']/area_salesperson_data['切换数']

In [49]:
# 每个销售名下的项目开通钱包数汇总
wallet_sum = data_salesperson.groupby('销售姓名')['累计有效钱包数'].sum()
wallet_sum_1 = pd.DataFrame({'销售姓名':wallet_sum.index,'累计有效钱包数':wallet_sum.values})
area_salesperson_data = area_salesperson_data.merge(wallet_sum_1,how='left',on='销售姓名')

In [50]:
# 每个销售名下这一周增长的钱包数汇总
week_increase_sum = data_salesperson.groupby('销售姓名')['周增长'].sum()
week_increase_sum_1 = pd.DataFrame({'销售姓名':week_increase_sum.index,'周增长':week_increase_sum.values})
area_salesperson_data = area_salesperson_data.merge(week_increase_sum_1,how='left',on='销售姓名')

In [51]:
# 每个销售名下项目累计支付的运单数汇总
waybill_num = data_salesperson.groupby('销售姓名')['支付运单数量'].sum()
waybill_num_1 = pd.DataFrame({'销售姓名':waybill_num.index,'支付运单数':waybill_num.values})
area_salesperson_data = area_salesperson_data.merge(waybill_num_1,how='left',on='销售姓名')

In [52]:
# 每个销售名下项目累计付运费到钱包数汇总
pay_to_wallet_num = data_salesperson.groupby('销售姓名')['付运费到钱包的运单数量'].sum()
pay_to_wallet_num_1 = pd.DataFrame({'销售姓名':pay_to_wallet_num.index,'付运费到钱包的运单数':pay_to_wallet_num.values})
area_salesperson_data = area_salesperson_data.merge(pay_to_wallet_num_1,how='left',on='销售姓名')

In [53]:
# 新增计算列
area_salesperson_data['支付比例'] = area_salesperson_data['付运费到钱包的运单数']/area_salesperson_data['支付运单数']

## 生成总部销售数据

In [54]:
# 每个销售名下开通钱包服务的项目数量
dept_num = data_salesperson.groupby('销售姓名')['企业ID'].count()
# 转换
dept_num_1 = pd.DataFrame({'销售姓名':dept_num.index,'开通钱包项目数':dept_num.values})
# 拼接
general_salesperson_data = general_salesperson.merge(dept_num_1,how='left',on='销售姓名')
# 新增计算列

general_salesperson_data['开通钱包项目率'] = general_salesperson_data['开通钱包项目数']/general_salesperson_data['切换数']

# 每个销售名下的项目开通钱包数汇总
wallet_sum = data_salesperson.groupby('销售姓名')['累计有效钱包数'].sum()
wallet_sum_1 = pd.DataFrame({'销售姓名':wallet_sum.index,'累计有效钱包数':wallet_sum.values})
general_salesperson_data = general_salesperson_data.merge(wallet_sum_1,how='left',on='销售姓名')

# 每个销售名下这一周增长的钱包数汇总
week_increase_sum = data_salesperson.groupby('销售姓名')['周增长'].sum()
week_increase_sum_1 = pd.DataFrame({'销售姓名':week_increase_sum.index,'周增长':week_increase_sum.values})
general_salesperson_data = general_salesperson_data.merge(week_increase_sum_1,how='left',on='销售姓名')

# 每个销售名下项目累计支付的运单数汇总
waybill_num = data_salesperson.groupby('销售姓名')['支付运单数量'].sum()
waybill_num_1 = pd.DataFrame({'销售姓名':waybill_num.index,'支付运单数':waybill_num.values})
general_salesperson_data = general_salesperson_data.merge(waybill_num_1,how='left',on='销售姓名')

# 每个销售名下项目累计付运费到钱包数汇总
pay_to_wallet_num = data_salesperson.groupby('销售姓名')['付运费到钱包的运单数量'].sum()
pay_to_wallet_num_1 = pd.DataFrame({'销售姓名':pay_to_wallet_num.index,'付运费到钱包数':pay_to_wallet_num.values})
general_salesperson_data = general_salesperson_data.merge(pay_to_wallet_num_1,how='left',on='销售姓名')

# 新增计算列
general_salesperson_data['支付比例'] = general_salesperson_data['付运费到钱包数']/general_salesperson_data['支付运单数']

## 【导出】总明细表/区域销售/总部销售表

In [ ]:
wallet_detail_file_location = "E:\\"+"好运宝钱包明细"+day+".xlsx"

In [55]:
from openpyxl import load_workbook

writer = pd.ExcelWriter(wallet_detail_file_location,engine='openpyxl')
book = load_workbook(writer.path)
writer.book = book
total_project.to_excel(excel_writer=writer,sheet_name='总明细',index=False)
area_salesperson_data.to_excel(excel_writer=writer,sheet_name='区域销售',index=False)
general_salesperson_data.to_excel(excel_writer=writer,sheet_name='总部销售',index=False)

writer.save()
writer.close()